
# Caricamento libri


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import re
import os

def estrai_contenuto_gutenberg(testo):
    pattern_start = r'\*\*\* START OF (?:THE |THIS )?PROJECT GUTENBERG EBOOK.*?\*\*\*'
    pattern_end = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK '

    match_start = re.search(pattern_start, testo, re.IGNORECASE | re.DOTALL)
    match_end = re.search(pattern_end, testo, re.IGNORECASE | re.DOTALL)

    if match_start and match_end:
        return testo[match_start.end():match_end.start()].strip()
    elif match_start:
        return testo[match_start.end():].strip()
    elif match_end:
        return testo[:match_end.start()].strip()
    else:
        return testo

def carica_tutti_i_libri(path_input, cartella_output):
    autori = ["primo autore", "secondo autore", "terzo autore"]

    # Crea cartella output se non esistente
    os.makedirs(cartella_output, exist_ok=True)

    libri_raggruppati = {autore: [] for autore in autori}

    for autore in autori:
        cartella = f"{path_input}{autore}"

        try:
            file_txt = [f for f in os.listdir(cartella)
                       if f.endswith('.txt')]

            if not file_txt:
                continue

            file_txt.sort()

            for nome_file in file_txt:
                percorso_completo = os.path.join(cartella, nome_file)

                try:
                    with open(percorso_completo, "r", encoding="utf-8") as f:
                        testo_completo = f.read()

                    contenuto_libro = estrai_contenuto_gutenberg(testo_completo)

                    if len(contenuto_libro) > 100:
                        libri_raggruppati[autore].append(contenuto_libro)

                except Exception:
                    continue

        except (FileNotFoundError, Exception):
            continue

    # Salva file combinati
    for autore, libri_autore in libri_raggruppati.items():
        if libri_autore:
            nome_file = f"{autore.replace(' ', '_')}_tutti_i_libri.txt"
            percorso_file = os.path.join(cartella_output, nome_file)

            with open(percorso_file, "w", encoding="utf-8") as f:
                f.write("\n\n" + "="*80 + "\n\n".join(libri_autore))

    return libri_raggruppati

# Esecuzione
path_trainings = "/content/drive/MyDrive/libri_training/"
cartella_output = "/content/drive/MyDrive/libri_puliti_training"

libri_raggruppati = carica_tutti_i_libri(path_trainings, cartella_output)

# Riepilogo essenziale
for autore, libri in libri_raggruppati.items():
    if libri:
        caratteri_totali = sum(len(libro) for libro in libri)
        print(f"{autore}: {len(libri)} libri, {caratteri_totali:,} caratteri")

primo autore: 5 libri, 2,274,448 caratteri
secondo autore: 7 libri, 3,173,605 caratteri
terzo autore: 8 libri, 2,208,855 caratteri


# Funzione di estrazione paragrafi


#Preparazione dati: funzioni varie

In [11]:
import re
import os
from sklearn.model_selection import train_test_split

def crea_corpus_word_embedding(
    cartella_libri_puliti_training,
    cartella_libri_puliti_test_val,
    cartella_output_finale,
    min_words_embedding=50,
    max_words_embedding=100,
    debug=False
):
    """
    Crea corpus unificato per word embedding FastText
    Include tutti i paragrafi di tutti gli autori (più permissivo sui limiti)
    """
    print(f"\n{'='*50}")
    print(f"CREAZIONE CORPUS per WORD EMBEDDING")
    print(f"{'='*50}")
    print(f"Parametri: {min_words_embedding}-{max_words_embedding} parole per paragrafo")

    # Cartella per i file di embedding
    cartella_embedding = os.path.join(cartella_output_finale, "word_embedding_corpus")
    os.makedirs(cartella_embedding, exist_ok=True)

    autori = ["primo autore", "secondo autore", "terzo autore"]

    # Lista per raccogliere tutti i paragrafi
    tutti_paragrafi = []
    paragrafi_per_autore = {}

    # Processa entrambe le cartelle (training + test/val)
    cartelle_source = [
        ("training", cartella_libri_puliti_training),
        ("test_val", cartella_libri_puliti_test_val)
    ]

    for tipo_set, cartella_source in cartelle_source:
        print(f"\n  Processando set {tipo_set}:")

        for autore in autori:
            nome_file_autore = f"{autore.replace(' ', '_')}_tutti_i_libri.txt"
            file_source = os.path.join(cartella_source, nome_file_autore)

            if os.path.exists(file_source):
                # Estrai paragrafi con limiti più permissivi
                paragrafi_autore = estrai_paragrafi_da_file_corretto(
                    file_source,
                    min_words=min_words_embedding,
                    max_words=max_words_embedding,
                    debug=debug
                )

                print(f"    {autore} ({tipo_set}): {len(paragrafi_autore)} paragrafi")

                # Aggiungi alla collezione totale
                tutti_paragrafi.extend(paragrafi_autore)

                # Tieni traccia per autore
                if autore not in paragrafi_per_autore:
                    paragrafi_per_autore[autore] = []
                paragrafi_per_autore[autore].extend(paragrafi_autore)
            else:
                print(f"    {autore} ({tipo_set}): File non trovato")

    # Salva corpus unificato per FastText
    print(f"\nSalvataggio corpus per word embedding:")

    # 1. Corpus completo (tutti gli autori insieme)
    file_corpus_completo = os.path.join(cartella_embedding, "corpus_completo.txt")
    with open(file_corpus_completo, "w", encoding="utf-8") as f:
        for paragrafo in tutti_paragrafi:
            # Ogni paragrafo su una riga (standard per FastText)
            f.write(paragrafo.replace('\n', ' ') + '\n')

    print(f"  ✓ Corpus completo: {len(tutti_paragrafi)} paragrafi → corpus_completo.txt")

    # 2. Corpus separati per autore (opzionale, utile per analisi)
    for autore, paragrafi in paragrafi_per_autore.items():
        nome_file_autore_corpus = f"corpus_{autore.replace(' ', '_')}.txt"
        file_corpus_autore = os.path.join(cartella_embedding, nome_file_autore_corpus)

        with open(file_corpus_autore, "w", encoding="utf-8") as f:
            for paragrafo in paragrafi:
                f.write(paragrafo.replace('\n', ' ') + '\n')

        print(f"  ✓ {autore}: {len(paragrafi)} paragrafi → {nome_file_autore_corpus}")

    # 3. Salva statistiche
    tutte_le_parole = []
    for paragrafo in tutti_paragrafi:
        parole = re.findall(r'\b\w+\b', paragrafo.lower())
        tutte_le_parole.extend(parole)

    vocabulary_unico = set(tutte_le_parole)

    file_stats = os.path.join(cartella_embedding, "corpus_statistics.txt")
    with open(file_stats, "w", encoding="utf-8") as f:
        f.write(f"STATISTICHE CORPUS WORD EMBEDDING\n")
        f.write(f"=====================================\n\n")
        f.write(f"Parametri estrazione: {min_words_embedding}-{max_words_embedding} parole/paragrafo\n\n")
        f.write(f"TOTALI:\n")
        f.write(f"  - Paragrafi: {len(tutti_paragrafi):,}\n")
        f.write(f"  - Parole totali: {len(tutte_le_parole):,}\n")
        f.write(f"  - Vocabulary unico: {len(vocabulary_unico):,}\n\n")
        f.write(f"PER AUTORE:\n")
        for autore, paragrafi in paragrafi_per_autore.items():
            f.write(f"  - {autore}: {len(paragrafi):,} paragrafi\n")

    print(f"\nStatistiche vocabulary:")
    print(f"  - Paragrafi totali: {len(tutti_paragrafi):,}")
    print(f"  - Parole totali: {len(tutte_le_parole):,}")
    print(f"  - Vocabulary unico: {len(vocabulary_unico):,}")

    return cartella_embedding

def conta_parole_correttamente(testo):
    """Conta le parole in modo accurato usando regex"""
    if not testo or not testo.strip():
        return 0

    # Rimuovi spazi extra e normalizza
    testo_pulito = re.sub(r'\s+', ' ', testo.strip())

    # Usa regex per trovare sequenze di caratteri alfanumerici
    parole = re.findall(r'\b\w+\b', testo_pulito)

    return len(parole)

def pulisci_paragrafo(paragrafo):
    """Pulisce un paragrafo mantenendo la leggibilità"""
    if not paragrafo:
        return ""

    # Rimuovi caratteri di controllo
    paragrafo = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', paragrafo)

    # Normalizza spazi multipli
    paragrafo = re.sub(r'\s+', ' ', paragrafo)

    # Rimuovi spazi all'inizio e fine
    paragrafo = paragrafo.strip()

    # Rimuovi linee che sono solo punteggiatura o numeri
    if re.match(r'^[^\w]*$', paragrafo) or re.match(r'^\d+\.?\s*$', paragrafo):
        return ""

    return paragrafo

def estrai_paragrafi_da_file_corretto(file_path, min_words=50, max_words=100, debug=False):
    """
    Estrae paragrafi validi da un singolo file con conteggio parole accurato
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            testo = f.read()
    except Exception as e:
        print(f"Errore leggendo {file_path}: {e}")
        return []

    if debug:
        print(f"File: {os.path.basename(file_path)}")
        print(f"Dimensione originale: {len(testo)} caratteri")

    # Gestisci diversi tipi di separatori di paragrafo
    # Prima normalizza i line break
    testo = testo.replace('\r\n', '\n').replace('\r', '\n')

    # Identifica paragrafi (linee vuote multiple = separatore paragrafo)
    paragrafi_raw = re.split(r'\n\s*\n', testo)

    if debug:
        print(f"Paragrafi raw trovati: {len(paragrafi_raw)}")

    paragrafi_validi = []
    stats = {'troppo_corti': 0, 'troppo_lunghi': 0, 'validi': 0, 'vuoti': 0}

    for i, paragrafo_raw in enumerate(paragrafi_raw):
        # Pulisci il paragrafo
        paragrafo = pulisci_paragrafo(paragrafo_raw)

        if not paragrafo:
            stats['vuoti'] += 1
            continue

        # Conta parole accuratamente
        num_parole = conta_parole_correttamente(paragrafo)

        if debug and i < 5:  # Mostra primi 5 per debug
            print(f"Paragrafo {i+1}: {num_parole} parole - '{paragrafo[:100]}...'")

        # Filtra per lunghezza
        if num_parole < min_words:
            stats['troppo_corti'] += 1
        elif num_parole > max_words:
            stats['troppo_lunghi'] += 1
        else:
            paragrafi_validi.append(paragrafo)
            stats['validi'] += 1

    if debug:
        print(f"Statistiche estrazione:")
        print(f"  - Paragrafi validi: {stats['validi']}")
        print(f"  - Troppo corti (<{min_words}): {stats['troppo_corti']}")
        print(f"  - Troppo lunghi (>{max_words}): {stats['troppo_lunghi']}")
        print(f"  - Vuoti/invalidi: {stats['vuoti']}")

    return paragrafi_validi

def verifica_paragrafi_estratti(paragrafi, min_words=50, max_words=100):
    """Verifica che i paragrafi estratti rispettino i limiti"""
    print(f"\nVerifica {len(paragrafi)} paragrafi:")

    problemi = []
    for i, paragrafo in enumerate(paragrafi):
        num_parole = conta_parole_correttamente(paragrafo)

        if num_parole < min_words:
            problemi.append(f"Paragrafo {i+1}: {num_parole} parole (< {min_words})")
        elif num_parole > max_words:
            problemi.append(f"Paragrafo {i+1}: {num_parole} parole (> {max_words})")

    if problemi:
        print(f"ATTENZIONE: {len(problemi)} paragrafi fuori range:")
        for problema in problemi[:10]:  # Mostra max 10
            print(f"  - {problema}")
        if len(problemi) > 10:
            print(f"  ... e altri {len(problemi) - 10}")
    else:
        print("Tutti i paragrafi rispettano i limiti di parole")

    # Statistiche
    parole_per_paragrafo = [conta_parole_correttamente(p) for p in paragrafi]
    if parole_per_paragrafo:
        print(f"Parole per paragrafo - Min: {min(parole_per_paragrafo)}, "
              f"Max: {max(parole_per_paragrafo)}, "
              f"Media: {sum(parole_per_paragrafo)/len(parole_per_paragrafo):.1f}")

def crea_struttura_dataset_finale_corretta(
    cartella_libri_puliti_training,
    cartella_libri_puliti_test_val,
    cartella_output_finale,
    min_words=50,
    max_words=100,
    test_size=0.5,
    debug=False,
    verifica_output=True,
    crea_embedding_corpus=False  # <-- AGGIUNGI QUESTO PARAMETRO

):
    """
    Versione corretta della creazione dataset con conteggio parole accurato
    """

    # Pulisci cartella output
    if os.path.exists(cartella_output_finale):
        print(f"Pulizia cartella output: {cartella_output_finale}")
        import shutil
        shutil.rmtree(cartella_output_finale)

   # AGGIUNGI QUESTO BLOCCO all'inizio della funzione (dopo la pulizia cartella)
    if crea_embedding_corpus:
        crea_corpus_word_embedding(
            cartella_libri_puliti_training,
            cartella_libri_puliti_test_val,
            cartella_output_finale,
            debug=debug
        )
        print(f"\n{'='*50}")
        print(f"CREAZIONE DATASET per CLASSIFICATION")
        print(f"{'='*50}")

    # Crea struttura
    sets = ["training_set", "test_set", "eval_set"]
    autori = ["primo autore", "secondo autore", "terzo autore"]

    for set_name in sets:
        for autore in autori:
            cartella_set_autore = os.path.join(cartella_output_finale, set_name, autore)
            os.makedirs(cartella_set_autore, exist_ok=True)

    print(f"Struttura cartelle creata in: {cartella_output_finale}")

    # TRAINING SET
    print(f"\nProcessing TRAINING SET (parole: {min_words}-{max_words}):")
    training_stats = {}

    for autore in autori:
        nome_file_autore = f"{autore.replace(' ', '_')}_tutti_i_libri.txt"
        file_source = os.path.join(cartella_libri_puliti_training, nome_file_autore)

        print(f"\n  {autore}:")

        if os.path.exists(file_source):
            # Estrai paragrafi con nuovo metodo
            paragrafi_autore = estrai_paragrafi_da_file_corretto(
                file_source,
                min_words=min_words,
                max_words=max_words,
                debug=debug
            )

            if paragrafi_autore:
                # Verifica se richiesto
                if verifica_output:
                    verifica_paragrafi_estratti(paragrafi_autore, min_words, max_words)

                # Salva paragrafi
                cartella_training_autore = os.path.join(cartella_output_finale, "training_set", autore)
                salva_paragrafi_separati(paragrafi_autore, cartella_training_autore, "par")

                training_stats[autore] = len(paragrafi_autore)
                print(f"    Training: {len(paragrafi_autore)} paragrafi salvati")
            else:
                print(f"    Nessun paragrafo valido estratto")
                training_stats[autore] = 0
        else:
            print(f"    File non trovato: {nome_file_autore}")
            training_stats[autore] = 0

    # TEST & EVAL SET
    print(f"\nProcessing TEST & EVAL SET:")
    test_eval_stats = {}

    for autore in autori:
        nome_file_autore = f"{autore.replace(' ', '_')}_tutti_i_libri.txt"
        file_source = os.path.join(cartella_libri_puliti_test_val, nome_file_autore)

        print(f"\n  {autore}:")

        if os.path.exists(file_source):
            # Estrai paragrafi
            paragrafi_autore = estrai_paragrafi_da_file_corretto(
                file_source,
                min_words=min_words,
                max_words=max_words,
                debug=debug
            )

            if paragrafi_autore:
                # Verifica
                if verifica_output:
                    verifica_paragrafi_estratti(paragrafi_autore, min_words, max_words)

                # Dividi in test e eval
                if len(paragrafi_autore) >= 2:
                    paragrafi_test, paragrafi_eval = train_test_split(
                        paragrafi_autore,
                        test_size=test_size,
                        random_state=42
                    )
                else:
                    # Se c'è solo 1 paragrafo, mettilo nel test
                    paragrafi_test = paragrafi_autore
                    paragrafi_eval = []

                # Salva
                cartella_test_autore = os.path.join(cartella_output_finale, "test_set", autore)
                cartella_eval_autore = os.path.join(cartella_output_finale, "eval_set", autore)

                salva_paragrafi_separati(paragrafi_test, cartella_test_autore, "par")
                salva_paragrafi_separati(paragrafi_eval, cartella_eval_autore, "par")

                test_eval_stats[autore] = {
                    'totale': len(paragrafi_autore),
                    'test': len(paragrafi_test),
                    'eval': len(paragrafi_eval)
                }

                print(f"    Totale: {len(paragrafi_autore)} → Test: {len(paragrafi_test)}, Eval: {len(paragrafi_eval)}")
            else:
                print(f"    Nessun paragrafo valido estratto")
                test_eval_stats[autore] = {'totale': 0, 'test': 0, 'eval': 0}
        else:
            print(f"    File non trovato: {nome_file_autore}")
            test_eval_stats[autore] = {'totale': 0, 'test': 0, 'eval': 0}

    # RIEPILOGO FINALE
    print(f"\nRIEPILOGO DATASET FINALE:")
    print(f"Parametri: parole per paragrafo {min_words}-{max_words}")

    # AGGIUNGI QUESTO messaggio se è stato creato il corpus
    if crea_embedding_corpus:
        print(f"\n✓ CORPUS WORD EMBEDDING creato in: word_embedding_corpus/")
        print(f"  → Usa 'corpus_completo.txt' per addestrare FastText")

    print(f"\nTRAINING SET:")
    for autore, count in training_stats.items():
        print(f"  {autore}: {count} paragrafi")

    print(f"\nTEST SET:")
    for autore, stats in test_eval_stats.items():
        print(f"  {autore}: {stats['test']} paragrafi")

    print(f"\nEVAL SET:")
    for autore, stats in test_eval_stats.items():
        print(f"  {autore}: {stats['eval']} paragrafi")

    print(f"\nTOTALE PER AUTORE:")
    for autore in autori:
        training_count = training_stats.get(autore, 0)
        test_count = test_eval_stats.get(autore, {}).get('test', 0)
        eval_count = test_eval_stats.get(autore, {}).get('eval', 0)
        totale = training_count + test_count + eval_count
        print(f"  {autore}: {totale} paragrafi totali")

    return cartella_output_finale

def salva_paragrafi_separati(paragrafi, cartella_output, prefisso="par"):
    """Salva ogni paragrafo come file separato"""
    os.makedirs(cartella_output, exist_ok=True)

    for i, paragrafo in enumerate(paragrafi, 1):
        nome_file = f"{prefisso}-{i:03d}.txt"  # Numerazione con zeri iniziali
        percorso_file = os.path.join(cartella_output, nome_file)

        with open(percorso_file, "w", encoding="utf-8") as f:
            f.write(paragrafo)

# TEST DELLA FUNZIONE
if __name__ == "__main__":
    # Esempio di uso
    print("Test conteggio parole:")

    testi_test = [
        "Questa è una frase con esattamente otto parole totali.",
        "Una   frase    con     spazi      multipli.",
        "Frase, con! punteggiatura? attaccata: alle; parole.",
        "   Spazi iniziali e finali   ",
        "",
        "Una sola parola"
    ]

    for testo in testi_test:
        parole = conta_parole_correttamente(testo)
        print(f"'{testo}' → {parole} parole")

In [12]:
dataset_path = crea_struttura_dataset_finale_corretta(
    cartella_libri_puliti_training="/content/drive/MyDrive/libri_puliti_training",
    cartella_libri_puliti_test_val="/content/drive/MyDrive/libri_test_val_puliti",
    cartella_output_finale="/content/drive/MyDrive/dataset_authorship_finale",
    min_words=50,
    max_words=100,
    debug=True,
    verifica_output=True,
    crea_embedding_corpus=True
)


CREAZIONE CORPUS WORD EMBEDDING (50-100 parole)

  Processando training:
    Paragrafo 2: 5 parole - '==================================================...'
    Paragrafo 3: 17 parole - 'BY R. AUSTIN FREEMAN Author of “The Singing Bone,”...'
    Stats - Validi: 1827, Corti: 5976, Lunghi: 1037, Vuoti: 9
    primo autore: 1827 paragrafi
    Paragrafo 2: 2 parole - '==================================================...'
    Paragrafo 3: 2 parole - 'BIG LAUREL...'
    Stats - Validi: 2125, Corti: 14505, Lunghi: 1436, Vuoti: 19
    secondo autore: 2125 paragrafi
    Paragrafo 2: 3 parole - '==================================================...'
    Paragrafo 3: 6 parole - 'INDIAN AND HUNTING STORIES FOR BOYS...'
    Stats - Validi: 1559, Corti: 6914, Lunghi: 1057, Vuoti: 21
    terzo autore: 1559 paragrafi

  Processando test_val:
    Paragrafo 1: 4 parole - 'Produced by Al Haines...'
    Paragrafo 2: 4 parole - 'The Eye of Osiris...'
    Paragrafo 3: 4 parole - 'A Detective Story by...'
 